In [25]:
!nvidia-smi

Sun Nov  3 23:11:09 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   42C    P0    65W / 300W |  27725MiB / 32478MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  On   | 00000000:08:00.0 Off |                    0 |
| N/A   40C    P0    51W / 300W |   1920MiB / 32478MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [26]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

import sys

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
import logging


logger = logging.getLogger('sequence_tagger_bert')

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

logger.handlers = []

fhandler = logging.handlers.TimedRotatingFileHandler(filename='logs.txt', when='midnight')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)

handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(formatter)
logger.addHandler(handler)

logger.setLevel(logging.DEBUG)

In [28]:
import torch

device = torch.device('cuda')
n_gpu = torch.cuda.device_count()

for i in range(n_gpu):
    print(torch.cuda.get_device_name(i))

Tesla V100-DGXS-32GB


In [29]:
CACHE_DIR = '../workdir/cache'
BATCH_SIZE = 16
#BATCH_SIZE = 8
PRED_BATCH_SIZE = 1000
MAX_LEN = 128
MAX_N_EPOCHS = 20
#MAX_N_EPOCHS = 100
#MAX_N_EPOCHS = 50
#MAX_N_EPOCHS = 10
REDUCE_ON_PLATEAU = False
WEIGHT_DECAY = 0.01
LEARNING_RATE = 3e-7
#LEARNING_RATE = 1e-5
#LEARNING_RATE = 2e-5

In [9]:
!ls /workspace/bert_sequence_tagger/src/data/NER/Varvara_v1/

dev_aspect.tsv	   test_aspect.tsv     train_aspect.tsv
dev_pred_full.tsv  test_pred_full.tsv  train_pred_full.tsv


In [30]:
from flair.datasets import ColumnCorpus


data_folder = '/workspace/bert_sequence_tagger/src/data/NER/Varvara_v3'
corpus = ColumnCorpus(data_folder, 
                      {0 : 'text', 1 : 'tag'},
                      train_file='train_pred_full.tsv',
                      test_file='test_pred_full.tsv',
                      dev_file='dev_pred_full.tsv')

print(corpus.obtain_statistics())

print(corpus.obtain_statistics())

2019-11-03 23:11:51,229 Reading data from /workspace/bert_sequence_tagger/src/data/NER/Varvara_v3
2019-11-03 23:11:51,230 Train: /workspace/bert_sequence_tagger/src/data/NER/Varvara_v3/train_pred_full.tsv
2019-11-03 23:11:51,230 Dev: /workspace/bert_sequence_tagger/src/data/NER/Varvara_v3/dev_pred_full.tsv
2019-11-03 23:11:51,231 Test: /workspace/bert_sequence_tagger/src/data/NER/Varvara_v3/test_pred_full.tsv
{
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 3077,
        "number_of_documents_per_class": {},
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 89350,
            "min": 6,
            "max": 106,
            "avg": 29.038024049398764
        }
    },
    "TEST": {
        "dataset": "TEST",
        "total_number_of_documents": 488,
        "number_of_documents_per_class": {},
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 13736,
            "min": 6,
       

In [31]:
a = corpus.make_tag_dictionary(tag_type = 'tag')

In [37]:
a.idx2item

[b'<unk>',
 b'O',
 b'B-OBJ',
 b'B-PREDFULL',
 b'I-PREDFULL',
 b'I-OBJ',
 b'<START>',
 b'<STOP>']

In [33]:
from bert_sequence_tagger import SequenceTaggerBert, BertForTokenClassificationCustom
from pytorch_transformers import BertTokenizer, BertForTokenClassification
import torch.nn as nn

from bert_sequence_tagger.bert_utils import make_bert_tag_dict_from_flair_corpus


bpe_tokenizer = BertTokenizer.from_pretrained('bert-base-cased', cache_dir=None, do_lower_case=False)

idx2tag, tag2idx = make_bert_tag_dict_from_flair_corpus(corpus)

model = nn.DataParallel(BertForTokenClassificationCustom.from_pretrained('bert-base-cased', cache_dir=None, num_labels=len(tag2idx))).cuda()
#model = BertForTokenClassification.from_pretrained('bert-base-cased', cache_dir=CACHE_DIR, num_labels=len(tag2idx)).cuda()

seq_tagger = SequenceTaggerBert(bert_model=model, bpe_tokenizer=bpe_tokenizer, idx2tag=idx2tag, tag2idx=tag2idx, max_len=200)

In [38]:
BATCH_SIZE = 10
PRED_BATCH_SIZE = 10
MAX_N_EPOCHS = 20
from torch.utils.data import RandomSampler, SequentialSampler

from bert_sequence_tagger.bert_utils import create_loader_from_flair_corpus, get_parameters_without_decay
from bert_sequence_tagger.model_trainer_bert import ModelTrainerBert

from pytorch_transformers import AdamW, WarmupLinearSchedule

from bert_sequence_tagger.metrics import f1_entity_level, f1_token_level

test_dataloader = create_loader_from_flair_corpus(corpus.test,
                                                  SequentialSampler,
                                                  batch_size=PRED_BATCH_SIZE)

_, __, test_metrics = seq_tagger.predict(test_dataloader, evaluate=True, 
                                         metrics=[f1_entity_level, f1_token_level])
logger.info(f'Entity-level f1: {test_metrics[1]}')
logger.info(f'Token-level f1: {test_metrics[2]}')

train_dataloader = create_loader_from_flair_corpus(corpus.train, 
                                                   RandomSampler, 
                                                   batch_size=BATCH_SIZE)
val_dataloader = create_loader_from_flair_corpus(corpus.dev,
                                                 SequentialSampler,
                                                 batch_size=PRED_BATCH_SIZE)

optimizer = AdamW(get_parameters_without_decay(model), lr=LEARNING_RATE, betas=(0.9, 0.999), 
                  eps =1e-6, weight_decay=0.01, correct_bias=True)
lr_scheduler = WarmupLinearSchedule(optimizer, warmup_steps=0.1, 
                                    t_total=(len(corpus.train) / BATCH_SIZE)*MAX_N_EPOCHS)
trainer = ModelTrainerBert(model=seq_tagger, 
                           optimizer=optimizer, 
                           lr_scheduler=lr_scheduler,
                           train_dataloader=train_dataloader, 
                           val_dataloader=val_dataloader,
                           update_scheduler='es',
                           keep_best_model=True,
                           restore_bm_on_lr_change=False,
                           max_grad_norm=1.,
                           validation_metrics=[f1_entity_level],
                           decision_metric=lambda metrics: -metrics[1])

trainer.train(epochs=MAX_N_EPOCHS)


test_dataloader = create_loader_from_flair_corpus(corpus.test,
                                                  SequentialSampler,
                                                  batch_size=PRED_BATCH_SIZE)

_, __, test_metrics = seq_tagger.predict(test_dataloader, evaluate=True, 
                                         metrics=[f1_entity_level, f1_token_level])
logger.info(f'Entity-level f1: {test_metrics[1]}')
logger.info(f'Token-level f1: {test_metrics[2]}')

2019-11-04 00:15:05,500 - sequence_tagger_bert - INFO - Entity-level f1: 0.5359922178988327
2019-11-04 00:15:05,502 - sequence_tagger_bert - INFO - Token-level f1: 0.5660714285714287





Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

2019-11-04 00:16:12,843 - sequence_tagger_bert - INFO - Train loss: 0.02296537380097071
2019-11-04 00:16:15,939 - sequence_tagger_bert - INFO - Validation loss: 0.5064210319641705
2019-11-04 00:16:15,940 - sequence_tagger_bert - INFO - Validation metrics: (0.4930847865303668,)
2019-11-04 00:16:15,965 - sequence_tagger_bert - INFO - Current learning rate: 0.0





Epoch:   5%|▌         | 1/20 [01:10<22:17, 70.38s/it]

2019-11-04 00:17:23,664 - sequence_tagger_bert - INFO - Train loss: 0.02211489531630348
2019-11-04 00:17:26,546 - sequence_tagger_bert - INFO - Validation loss: 0.5068650401946975
2019-11-04 00:17:26,547 - sequence_tagger_bert - INFO - Validation metrics: (0.5014961101137044,)
2019-11-04 00:17:26,565 - sequence_tagger_bert - INFO - Current learning rate: 0.0


...

2019-11-04 00:37:24,729 - sequence_tagger_bert - INFO - Train loss: 0.019374078425409443
2019-11-04 00:37:27,703 - sequence_tagger_bert - INFO - Validation loss: 0.5228917853106041
2019-11-04 00:37:27,704 - sequence_tagger_bert - INFO - Validation metrics: (0.5023866348448688,)
2019-11-04 00:37:27,705 - sequence_tagger_bert - INFO - Current learning rate: 0.0





Epoch:  95%|█████████▌| 19/20 [22:22<01:11, 71.99s/it]

2019-11-04 00:38:37,752 - sequence_tagger_bert - INFO - Train loss: 0.019024234159634897
2019-11-04 00:38:41,119 - sequence_tagger_bert - INFO - Validation loss: 0.5231161052730252
2019-11-04 00:38:41,121 - sequence_tagger_bert - INFO - Validation metrics: (0.5014925373134328,)
2019-11-04 00:38:41,122 - sequence_tagger_bert - INFO - Current learning rate: 0.0





Epoch: 100%|██████████| 20/20 [23:35<00:00, 72.42s/it]




2019-11-04 00:38:45,008 - sequence_tagger_bert - INFO - Entity-level f1: 0.5389105058365758
2019-11-04 00:38:45,010 - sequence_tagger_bert - INFO - Token-level f1: 0.5688809629959874


In [34]:
#BATCH_SIZE = 10
#PRED_BATCH_SIZE = 10
#MAX_N_EPOCHS = 20
from torch.utils.data import RandomSampler, SequentialSampler

from bert_sequence_tagger.bert_utils import create_loader_from_flair_corpus, get_parameters_without_decay
from bert_sequence_tagger.model_trainer_bert import ModelTrainerBert

from pytorch_transformers import AdamW, WarmupLinearSchedule

from bert_sequence_tagger.metrics import f1_entity_level, f1_token_level

for LEARNING_RATE in [3e-6, 3e-7, 3e-4, 1e-5]:

        train_dataloader = create_loader_from_flair_corpus(corpus.train, 
                                                           RandomSampler, 
                                                           batch_size=BATCH_SIZE)
        val_dataloader = create_loader_from_flair_corpus(corpus.dev,
                                                         SequentialSampler,
                                                         batch_size=PRED_BATCH_SIZE)

        optimizer = AdamW(get_parameters_without_decay(model), lr=LEARNING_RATE, betas=(0.9, 0.999), 
                          eps =1e-6, weight_decay=0.01, correct_bias=True)
        lr_scheduler = WarmupLinearSchedule(optimizer, warmup_steps=0.1, 
                                            t_total=(len(corpus.train) / BATCH_SIZE)*MAX_N_EPOCHS)
        trainer = ModelTrainerBert(model=seq_tagger, 
                                   optimizer=optimizer, 
                                   lr_scheduler=lr_scheduler,
                                   train_dataloader=train_dataloader, 
                                   val_dataloader=val_dataloader,
                                   update_scheduler='es',
                                   keep_best_model=True,
                                   restore_bm_on_lr_change=False,
                                   max_grad_norm=1.,
                                   validation_metrics=[f1_entity_level],
                                   decision_metric=lambda metrics: -metrics[1])

        trainer.train(epochs=MAX_N_EPOCHS)


        test_dataloader = create_loader_from_flair_corpus(corpus.test,
                                                          SequentialSampler,
                                                          batch_size=PRED_BATCH_SIZE)

        _, __, test_metrics = seq_tagger.predict(test_dataloader, evaluate=True, 
                                                 metrics=[f1_entity_level, f1_token_level])
        logger.info(f'Entity-level f1: {test_metrics[1]}')
        logger.info(f'Token-level f1: {test_metrics[2]}')


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

2019-11-03 23:19:57,164 - sequence_tagger_bert - INFO - Train loss: 0.4787870369504153
2019-11-03 23:20:00,145 - sequence_tagger_bert - INFO - Validation loss: 0.28253188729286194
2019-11-03 23:20:00,146 - sequence_tagger_bert - INFO - Validation metrics: (0.3241469816272966,)
2019-11-03 23:20:00,166 - sequence_tagger_bert - INFO - Current learning rate: 0.0



Epoch:   5%|▌         | 1/20 [00:56<17:50, 56.34s/it]

2019-11-03 23:20:53,284 - sequence_tagger_bert - INFO - Train loss: 0.10299811927225305
2019-11-03 23:20:56,507 - sequence_tagger_bert - INFO - Validation loss: 0.32408687472343445
2019-11-03 23:20:56,508 - sequence_tagger_bert - INFO - Validation metrics: (0.4444444444444445,)
2019-11-03 23:20:56,532 - sequence_tagger_bert - INFO - Current learning rate: 0.0



Epoch:  10%|█         | 2/20 [01:52<16:54, 56.35s/it]

...

2019-11-03 23:38:04,720 - sequence_tagger_bert - INFO - Entity-level f1: 0.5571428571428572
2019-11-03 23:38:04,721 - sequence_tagger_bert - INFO - Token-level f1: 0.5844269466316709



Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

2019-11-03 23:38:58,046 - sequence_tagger_bert - INFO - Train loss: 0.025655677117445926
2019-11-03 23:39:01,038 - sequence_tagger_bert - INFO - Validation loss: 0.4682237505912781
2019-11-03 23:39:01,039 - sequence_tagger_bert - INFO - Validation metrics: (0.4930681133212779,)
2019-11-03 23:39:01,065 - sequence_tagger_bert - INFO - Current learning rate: 0.0



Epoch:   5%|▌         | 1/20 [00:56<17:48, 56.26s/it]

KeyboardInterrupt: 

In [39]:
with open('labels_v3_lr7.txt', 'w') as f:
    for _string in _:
        #f.seek(0)
        f.write(', '.join(_string) + '\n')

## отвалидировалась на существующих метриках, лучше при lr 3e-6, 3e-7

In [23]:
from torch.utils.data import RandomSampler, SequentialSampler

from bert_sequence_tagger.bert_utils import create_loader_from_flair_corpus, get_parameters_without_decay
from bert_sequence_tagger.model_trainer_bert import ModelTrainerBert

from pytorch_transformers import AdamW, WarmupLinearSchedule

from bert_sequence_tagger.metrics import f1_entity_level, f1_token_level

train_dataloader = create_loader_from_flair_corpus(corpus.train, 
                                                           RandomSampler, 
                                                           batch_size=BATCH_SIZE)
val_dataloader = create_loader_from_flair_corpus(corpus.dev,
                                                 SequentialSampler,
                                                 batch_size=PRED_BATCH_SIZE)

optimizer = AdamW(get_parameters_without_decay(model), lr=LEARNING_RATE, betas=(0.9, 0.999), 
                  eps =1e-6, weight_decay=0.01, correct_bias=True)
lr_scheduler = WarmupLinearSchedule(optimizer, warmup_steps=0.1, 
                                    t_total=(len(corpus.train) / BATCH_SIZE)*MAX_N_EPOCHS)
trainer = ModelTrainerBert(model=seq_tagger, 
                           optimizer=optimizer, 
                           lr_scheduler=lr_scheduler,
                           train_dataloader=train_dataloader, 
                           val_dataloader=val_dataloader,
                           update_scheduler='es',
                           keep_best_model=True,
                           restore_bm_on_lr_change=False,
                           max_grad_norm=1.,
                           validation_metrics=[f1_entity_level],
                           decision_metric=lambda metrics: -metrics[1])

trainer.train(epochs=MAX_N_EPOCHS)


test_dataloader = create_loader_from_flair_corpus(corpus.test,
                                                  SequentialSampler,
                                                  batch_size=PRED_BATCH_SIZE)

preds, probs, test_metrics = seq_tagger.predict(test_dataloader, evaluate=True, 
                                         metrics=[f1_entity_level, f1_token_level])
logger.info(f'Entity-level f1: {test_metrics[1]}')
logger.info(f'Token-level f1: {test_metrics[2]}')


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

2019-10-23 17:36:07,138 - sequence_tagger_bert - INFO - Train loss: 0.44207449675415433
2019-10-23 17:36:09,913 - sequence_tagger_bert - INFO - Validation loss: 0.19390782713890076
2019-10-23 17:36:09,914 - sequence_tagger_bert - INFO - Validation metrics: (0.5417457305502846,)
2019-10-23 17:36:09,933 - sequence_tagger_bert - INFO - Current learning rate: 0.0



Epoch:   5%|▌         | 1/20 [00:49<15:37, 49.32s/it]

2019-10-23 17:37:00,264 - sequence_tagger_bert - INFO - Train loss: 0.09446393226471422
2019-10-23 17:37:03,435 - sequence_tagger_bert - INFO - Validation loss: 0.2510414719581604
2019-10-23 17:37:03,438 - sequence_tagger_bert - INFO - Validation metrics: (0.389370306181398,)
2019-10-23 17:37:03,438 - sequence_tagger_bert - INFO - Current learning rate: 0.0



Epoch:  10%|█         | 2/20 [01:42<15:10, 50.58s/it]

...

2019-10-23 17:52:00,266 - sequence_tagger_bert - INFO - Train loss: 0.022491128675900904
2019-10-23 17:52:02,964 - sequence_tagger_bert - INFO - Validation loss: 0.4055901765823364
2019-10-23 17:52:02,965 - sequence_tagger_bert - INFO - Validation metrics: (0.44189852700491,)
2019-10-23 17:52:02,966 - sequence_tagger_bert - INFO - Current learning rate: 0.0



Epoch:  95%|█████████▌| 19/20 [16:42<00:52, 52.61s/it]

2019-10-23 17:52:52,142 - sequence_tagger_bert - INFO - Train loss: 0.022022977412353813
2019-10-23 17:52:55,384 - sequence_tagger_bert - INFO - Validation loss: 0.4082149267196655
2019-10-23 17:52:55,386 - sequence_tagger_bert - INFO - Validation metrics: (0.4378762999452655,)
2019-10-23 17:52:55,387 - sequence_tagger_bert - INFO - Current learning rate: 0.0



Epoch: 100%|██████████| 20/20 [17:34<00:00, 52.55s/it]


2019-10-23 17:52:59,515 - sequence_tagger_bert - INFO - Entity-level f1: 0.5282833251352681
2019-10-23 17:52:59,516 - sequence_tagger_bert - INFO - Token-level f1: 0.4879333040807372


In [ ]:
', '.join(preds[5])

In [ ]:
len(preds)

In [16]:
with open('labels_v.txt', 'w') as f:
    for _string in preds:
        #f.seek(0)
        f.write(', '.join(_string) + '\n')

In [29]:
def prepare_flair_corpus(corpus, name='tag', filter_tokens={'-DOCSTART-'}):
    result = []
    for sent in corpus[:10]:
        print ("sent", sent)
        print ("sent[0].text", sent[0].text)
        if sent[0].text in filter_tokens:
            continue
        else:
            result.append(([token.text for token in sent.tokens],
                           [token.tags[name].value for token in sent.tokens]))
    
    return result


In [17]:
test_dataloader = create_loader_from_flair_corpus(corpus.test,
                                                  SequentialSampler,
                                                  batch_size=PRED_BATCH_SIZE)

_, __, test_metrics = seq_tagger.predict(test_dataloader, evaluate=True, 
                                         metrics=[f1_entity_level, f1_token_level])
logger.info(f'Entity-level f1: {test_metrics[1]}')
logger.info(f'Token-level f1: {test_metrics[2]}')

2019-10-23 12:13:32,709 - sequence_tagger_bert - INFO - Entity-level f1: 0.43243243243243246
2019-10-23 12:13:32,711 - sequence_tagger_bert - INFO - Token-level f1: 0.43655723158828746


In [ ]:
(0.9143007822800387, 0.9306361914074436)

In [ ]:
class EvaluatorBase():
    """EvaluatorBase is abstract base class for all evaluators"""
    def get_evaluation_score_train_dev_test(self, tagger, datasets_bank, batch_size=-1):
        if batch_size == -1:
            batch_size = tagger.batch_size
        score_train, _ = self.predict_evaluation_score(tagger=tagger,
                                                       word_sequences=datasets_bank.word_sequences_train,
                                                       targets_tag_sequences=datasets_bank.tag_sequences_train,
                                                       batch_size=batch_size)
        score_dev, _ = self.predict_evaluation_score(tagger=tagger,
                                                     word_sequences=datasets_bank.word_sequences_dev,
                                                     targets_tag_sequences=datasets_bank.tag_sequences_dev,
                                                     batch_size=batch_size)
        score_test, msg_test = self.predict_evaluation_score(tagger=tagger,
                                                             word_sequences=datasets_bank.word_sequences_test,
                                                             targets_tag_sequences=datasets_bank.tag_sequences_test,
                                                             batch_size=batch_size)
        return score_train, score_dev, score_test, msg_test

    def predict_evaluation_score(self, tagger, word_sequences, targets_tag_sequences, batch_size):
        outputs_tag_sequences = tagger.predict_tags_from_words(word_sequences, batch_size)
        return self.get_evaluation_score(targets_tag_sequences, outputs_tag_sequences, word_sequences)

In [ ]:
class EvaluatorF1MacroTokenLevel(EvaluatorBase):
    def __init__(self):
        self.tag_list = None
        self.tag2idx = dict()

    def __init_tag_list(self, targets_tag_sequences):
        if self.tag_list is not None:
            return
        self.tag_list = list()
        for tag_seq in targets_tag_sequences:
            for t in tag_seq:
                if t not in self.tag_list:
                    self.tag_list.append(t)
                    self.tag2idx[t] = len(self.tag_list)
        self.tag_list.sort()

    def tag_seq_2_idx_list(self, tag_seq):
        return [self.tag2idx[t] for t in tag_seq]

    def __get_zeros_tag_dict(self):
        return {tag: 0 for tag in self.tag_list}

    def __add_dict(self, dict1, dict2):
        for tag in self.tag_list:
            dict1[tag] += dict2[tag]
        return dict1

    def __div_dict(self, dict, d):
        for tag in self.tag_list:
            dict[tag] /= d
        return dict

    def __get_M_F1_msg(self, F1, precision, recall):
        msg = '\nF1 scores\n'
        msg += '-' * 24 + '\n'
        sum_M_F1 = 0
        sum_precision = 0
        sum_recall = 0
        for tag in self.tag_list:
            sum_M_F1 += F1[tag]
            sum_precision += precision[tag]
            sum_recall += recall[tag]
            msg += '%15s = f1 = %1.2f, precision = %1.2f, recall = %1.2f\n' % (tag, F1[tag], precision[tag], recall[tag])
        M_F1 = sum_M_F1 / len(F1)
        M_PR = sum_precision / len(F1)
        M_RE = sum_recall / len(F1)
        msg += '-'*24 + '\n'
        msg += 'Macro-F1 = %1.3f' % M_F1
        msg += 'Macro-Prescion = %1.3f' % M_PR
        msg += 'Macro-Recall = %1.3f' % M_RE
        return M_F1, msg

    def __add_to_dict(self, dict_in, tag, val):
        if tag in dict_in:
            dict_in[tag] += val
        else:
            dict_in[tag] = val
        return dict_in

    """EvaluatorF1MacroTagComponents is macro-F1 scores evaluator for each class of BOI-like tags."""
    def get_evaluation_score(self, targets_tag_sequences, outputs_tag_sequences, word_sequences=None):
        # Create list of tags
        self.__init_tag_list(targets_tag_sequences)
        i = 0
        for elem in zip(targets_tag_sequences, outputs_tag_sequences):
            if (i < 4):
                i = i +1
                print (elem[0])
                print (elem[1])
        # Init values
        TP = self.__get_zeros_tag_dict()
        FP = self.__get_zeros_tag_dict()
        FN = self.__get_zeros_tag_dict()
        F1 = self.__get_zeros_tag_dict()
        precision = self.__get_zeros_tag_dict()
        recall = self.__get_zeros_tag_dict()
        for targets_seq, outputs_tag_seq in zip(targets_tag_sequences, outputs_tag_sequences):
            for t, o in zip(targets_seq, outputs_tag_seq):
                if t == o:
                    TP = self.__add_to_dict(TP, t, 1)
                else:
                    FN = self.__add_to_dict(FN, t, 1)
                    FP = self.__add_to_dict(FP, o, 1)
        # Calculate F1 for each tag
        for tag in self.tag_list:
            F1[tag] = (2 * TP[tag] / max(2 * TP[tag] + FP[tag] + FN[tag], 1)) * 100
            precision[tag] = (TP[tag] / max(TP[tag] + FP[tag], 1))*100
            recall[tag] = (TP[tag] / max(TP[tag] + FN[tag], 1))*100
        # Calculate Macro-F1 score and prepare the message
        M_F1, msg = self.__get_M_F1_msg(F1,precision, recall)
        print(msg)
        #self.validate_M_F1_scikitlearn( targets_tag_sequences, outputs_tag_sequences)
        return M_F1, msg
